In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import random
import cv2
import os
import numpy as np

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/ibtd.zip

Archive:  /content/drive/MyDrive/ibtd.zip
  inflating: ibtd/0000.jpg           
  inflating: ibtd/0001.jpg           
  inflating: ibtd/0002.jpg           
  inflating: ibtd/0003.jpg           
  inflating: ibtd/0004.jpg           
  inflating: ibtd/0005.jpg           
  inflating: ibtd/0006.jpg           
  inflating: ibtd/0007.jpg           
  inflating: ibtd/0008.jpg           
  inflating: ibtd/0009.jpg           
  inflating: ibtd/0010.jpg           
  inflating: ibtd/0011.jpg           
  inflating: ibtd/0012.jpg           
  inflating: ibtd/0013.jpg           
  inflating: ibtd/0014.jpg           
  inflating: ibtd/0015.jpg           
  inflating: ibtd/0016.jpg           
  inflating: ibtd/0017.jpg           
  inflating: ibtd/0018.jpg           
  inflating: ibtd/0019.jpg           
  inflating: ibtd/0020.jpg           
  inflating: ibtd/0021.jpg           
  inflating: ibtd/0022.jpg           
  inflating: ibtd/0023.jpg           
  inflating: ibtd/0024.jpg           
  inflat

In [ ]:
!rm /content/ibtd/Thumbs.db
!rm /content/ibtd/Mask/Thumbs.db
!rm -rf `find -type d -name .ipynb_checkpoints

/bin/bash: -c: line 0: unexpected EOF while looking for matching ``'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [ ]:
# ​save pixel numbers of skin and non-skin pixels
skinPixelNumber = [[[0 for k in range(256)] for j in range(256)] for i in range(256)]
nonskinPixelNumber = [[[0 for k in range(256)] for j in range(256)] for i in range(256)]
skin = 0
nonSkin = 0

In [ ]:
def take_input():
    images = []
    mask_images = os.listdir("/content/ibtd/Mask")
    mask_images = random.sample(mask_images, 555)
    for i in range(555):
        tempList = mask_images[i].split(".")
        temp = tempList[0] + ".jpg"
        images.append(temp)
    return images, mask_images


nonSkinPixels = 0
skinPixels = 0

In [ ]:
def train(images, mask_images):
    global skinPixels, nonSkinPixels
    for i in range(0, len(images) - 56):
        # print("eita normal image ", images[i], "ar eita masked ",mask_images[i])
        image = cv2.imread("/content/ibtd/" + images[i])
        mask_image = cv2.imread("/content/ibtd/Mask/" + mask_images[i])
        width, height, _ = image.shape
        # print(width, " ", height)

        for h in range(height):
            for w in range(width):
                blue = image[w][h][0]
                green = image[w][h][1]
                red = image[w][h][2]
                maskBlue = mask_image[w][h][0]
                maskGreen = mask_image[w][h][1]
                maskRed = mask_image[w][h][2]
                if maskRed > 250 and maskGreen > 250 and maskBlue > 250:
                    nonskinPixelNumber[red][green][blue] += 1
                    # print("#")
                    nonSkinPixels += 1
                else:
                    skinPixelNumber[red][green][blue] += 1
                    # print(".")
                    skinPixels += 1

    #print(nonSkinPixels, " ", skinPixels)

In [ ]:
# # write final probability of skin/non-skin to file
def write_ratio(T):
    #print(nonSkinPixels, " ", skinPixels)
    fp = open("/content/ibtd/sharif.txt", "w")
    for r in range(256):
        for g in range(256):
            for b in range(256):
                if skinPixels != 0 and nonSkinPixels != 0:
                    # print(".")
                    skin = skinPixelNumber[r][g][b] / skinPixels  # P(C|S)
                    nonSkin = nonskinPixelNumber[r][g][b] / nonSkinPixels  # P(C|NS)
                    if nonSkin != 0:
                        T = skin / nonSkin
                    else:
                        T = 0.0
                # print(skinPixelNumber[r][g][b]," ",nonskinPixelNumber[r][g][b])
                fp.write(f"{T}\n")
                # print(T)
    fp.close()

In [ ]:
def calculate_accuracy():
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    trained_value = np.zeros(shape=(256, 256, 256))
    sharif = open("/content/ibtd/sharif.txt", "r")
    for i in range(256):
        for j in range(256):
            for k in range(256):
                val = sharif.readline()
                trained_value[i][j][k] = float(val)

    # print(trained_value)
    for i in range(500, len(images) - 1):
        new_img = cv2.imread("/content/ibtd/" + images[i])
        new_mask_img = cv2.imread("/content/ibtd/Mask/" + mask_images[i])
        height, width, _ = new_img.shape
        for x in range(height):
            for y in range(width):
                red = new_img[x, y, 0]
                green = new_img[x, y, 1]
                blue = new_img[x, y, 2]
                if trained_value[red, green, blue] <= 0.4: #non skin hoile
                    new_img[x, y, 0] = 255
                    new_img[x, y, 1] = 255
                    new_img[x, y, 2] = 255
                    if new_img[x, y, 0] == new_mask_img[x, y, 0]:
                        tn += 1
                    else:
                        fn += 1
                else:
                    if new_img[x, y, 0] == new_mask_img[x, y, 0]:
                        tp += 1
                    else:
                        fp += 1
    res = 0
    hor = tp + tn
    lob = tp + fp + tn + fn
    print('True positive ',tp, "True Negative ", tn, "False Positive ", fp, "False Negative ", fn)
    if lob != 0:
        res = hor / lob
    print('Accuracy is: ',res)


In [ ]:
images = []
mask_images = []
for i in range(10):
    images, mask_images = take_input()
    train(images,mask_images)
    write_ratio(0.0)
    calculate_accuracy()

True positive  40645 True Negative  8412687 False Positive  244935 False Negative  2229540
Accuracy is:  0.7735616121331572
True positive  6633 True Negative  6444701 False Positive  88852 False Negative  1871612
Accuracy is:  0.7669387686199788
True positive  26287 True Negative  6940668 False Positive  69684 False Negative  2476182
Accuracy is:  0.7323752859430447
True positive  53691 True Negative  7904337 False Positive  283272 False Negative  2266737
Accuracy is:  0.7573277482749633
True positive  16196 True Negative  6047909 False Positive  156712 False Negative  2054383
Accuracy is:  0.7328046452049497
True positive  11580 True Negative  6153034 False Positive  120157 False Negative  1894326
Accuracy is:  0.7537034956303856


KeyboardInterrupt: ignored